# [실습] OpenAI API와 Tool Calling을 이용한 뉴스 요약 봇 만들기
- OpenAI API를 통해 OpenAI의 기능을 호출하고 활용

## 환경 구성하기

In [1]:
# openai API 라이브러리 설치
!pip install openai tiktoken --upgrade

  Attempting uninstall: openai
    Found existing installation: openai 1.61.0
    Uninstalling openai-1.61.0:
      Successfully uninstalled openai-1.61.0



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 라이브러리 로드
import openai
import os

# openai api key 설정
os.environ['OPENAI_API_KEY']  = ""
client = openai.OpenAI()

# API키 검증하기
try : client.models.lis(); print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")
except: print(f"OPENAI_API_KEY가 유효하지 않습니다.")

OPENAI_API_KEY가 유효하지 않습니다.


### 1. LLM으로 웹 크롤링 코드 구현 프롬프트 만들기
  - 네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드
  - 검색어가 query인수로 주어지면, 관련도순으로 검색된 뉴스기사 30개의 정보를 전달하는 함수를 요청
  - 네이버 뉴스 API를 사용한다는 내용을 명시하면 정확도가 상승함

In [ ]:
prompt = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 'query'인수로 주어지면, 관련도순으로 검색된 
뉴스 기사 30개의 정보를 아래 포맷의 문자열로 변환하는 함수 get_news(query)를 만들어 주세요.
---
제목 : (뉴스 제목)
URL : (뉴스 링크)
내용 : (뉴스 내용)
---


# 검색어
query  ='생성형 AI'

# 검색 결과를 return하는 함수
def get_news(query):
...

result = get_news(query)
print(result)

'''

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages=[
        {
            "role":'system',
            'content' :'당신은 파이썬 코딩의 전문가입니다. 설명 없이 코드만 출력해주세요.'
        },
        {
            'role' : 'user',
            'content' : prompt
        }
    ]
)

In [ ]:
import requests

# 네이버 개발자 센터에서 발급받은 Client ID와 Client Secret을 입력합니다.
CLIENT_ID = 'your_client_id'  # 본인의 Client ID로 변경
CLIENT_SECRET = 'your_client_secret'  # 본인의 Client Secret으로 변경

# 검색어(query)를 입력받아 관련 뉴스 30개를 가져오는 함수
def get_news(query):
    url = "https://openapi.naver.com/v1/search/news.json"
    
    params = {
        'query': query,  # 검색어
        'display': 30,    # 검색 결과 수 (최대 30개)
        'sort': 'sim'     # 관련도순으로 정렬 (기본값은 'sim', 관련도순)
    }
    
    headers = {
        'X-Naver-Client-Id': CLIENT_ID,
        'X-Naver-Client-Secret': CLIENT_SECRET
    }
    
    # 네이버 API 호출
    response = requests.get(url, headers=headers, params=params)
    
    # 응답이 성공적이면
    if response.status_code == 200:
        news_items = response.json().get('items', [])
        
        # 뉴스 기사 정보를 포맷에 맞게 출력
        result = ""
        for item in news_items:
            title = item['title'].replace('<b>', '').replace('</b>', '')  # 제목에서 HTML 태그 제거
            link = item['link']
            description = item['description'].replace('<b>', '').replace('</b>', '')  # 내용에서 HTML 태그 제거
            
            result += f"제목 : {title}\nURL : {link}\n내용 : {description}\n---\n"
        
        return result
    else:
        return f"Error: {response.status_code}"

# 검색어 예시
query = '생성형 AI'
result = get_news(query)
print(result)

### OpenAI API를 이용하여 요약하기
   - 실제로는 각각의 URL에 접속해서 뉴스 링크를 가져오는 것이 일반적이지만, 간단하게 검색 결과만을 활용한는 코드를 만든다.
   - query에 대한 뉴스 검색 결과를 요약